# Clothing Classification with Convolutional Neural Networks
**Purpose**:  
Build and train a convolutional neural network (CNN) to classify images of clothing. This model is trained on 60,000 images that include 10 types of articles of clothing.  

This project expands on the previous investigation into classifying clothing using neural networks, except we are now using convolutions for higher performance.

Project based on [TensorFlow's classification example](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l04c01_image_classification_with_cnns.ipynb)



**Install and import dependencies**

In [ ]:
import tensorflow as tf

In [ ]:
# import tensorflow datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

**Import the Fashion MNIST dataset**

In [ ]:
# import the dataset from TensorFlow using the Datasets API
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
# store the class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [ ]:
# find the number of training and testing images
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

**Preprocess the data**

In [ ]:
# normalize the values of the pixels from the range [0,255] to [0,1]
# create a normalization function and apply it to the datasets
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# cache the datasets to keep them in memory after first load from disk
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()


**Build the model**  

Configure the layers and compile the model.  
Layers:

*   **"convolutions":** Network starts with two pairs of `Conv2D`/`MaxPool`, with the first layer of Conv2D filters being applied to the input image and creating 32 output (convoluted images). The 32 outputs are then reduced in size using a MaxPooling of (2,2). The second layer of Conv2D filters uses a (3,3) kernel takes the 32 convoluted images and creates 64 outputs.
*   **output-Dense:** A 128-neuron, followed by 10 node softmax layer, with each node representing a class of clothing



In [ ]:
# set up the layers by chaining two together

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2,2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
# compile the model adding the loss function, optimizer, and metrics
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

**Train the model**  
1. Define the iteration behavior
- Repeat until specified epochs
- Shuffle the order of examples
- Use batches of 32 variables
2. Train using the `model.fit` method
- Feed the model the training data
- Model learning go brr
- Epochs limit training to 5 full iterations

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

**Evaluate accuracy**  
Accuracy on test_dataset is lower than accuracy on training, but this is normal! We can expect performance to go down.

In [ ]:
# compare how the model performs on the test dataset to the training data

test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

**Make predictions**  
Make some predictions about images

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

predictions.shape

In [ ]:
# full probability info
predictions[0]

In [ ]:
# see which label has the highest confidence value
np.argmax(predictions[0])

In [ ]:
test_labels[0]

In [ ]:
# look at full set of class predictions
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)
  

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])   
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0,1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')
  

In [ ]:
# look at the 0th image, predictions, and prediction array

i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# plot the first X test images, their predicted label, and the true label
# color correct predictions in blue, incorrect predictions in red to give
# percent for the predicted label

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)

In [ ]:
# use the trained model to make a prediction about a single image
img = test_images[0]
print(img.shape)

# add image to a batch where it's the only member since tf.keras models are
# optimized to make predictions on a batch/collection of examples at once
img = np.array([img])
print(img.shape)

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

In [ ]:
np.argmax(predictions_single[0])